In [1]:
import os
import glob
import shutil
import hashlib
from collections import OrderedDict
import pandas as pd
import time
import pickle


In [3]:

HEADER_ROW=['bag', 'time_sec', 'time_nsec', 'GPS', 'v_ego', 'key_event', 'frame_name']
frames_dir = '/media/dean/deans_data/construction_frames/'
bag_dirs = os.listdir(frames_dir)
save_dir = os.path.join(frames_dir,'kache_set')
os.makedirs(save_dir, 0o755 , exist_ok = True )

img_data = []
lookup_table = {}

for bag in bag_dirs:
    if os.path.join(frames_dir,bag) != save_dir:
        img_dict = {}
        bag_path = os.path.join(frames_dir, bag)
        img_dict['bag_path'] = bag_path
        img_dict['frames'] = []
        img_dict['csv_path'] = os.path.join(bag_path, 'frames.csv')
        csv_logger = pd.read_csv(img_dict['csv_path'], names=HEADER_ROW, skiprows=1)

        # Get all images
        imgs = glob.glob(os.path.join(bag_path,'**/', '*.jpg'))

        for img in imgs:
            d = {}
            d['frame_path'] = img
            new_path = os.path.split(os.path.split(os.path.split(img)[0])[0])[1].rstrip('/')+'-'+os.path.split(img)[1]
            hash_object = hashlib.sha1(str.encode(new_path))
            hex_dig = hash_object.hexdigest()
            #shutil.copyfile(img, os.path.join(save_dir, str(hex_dig)+'.jpg'))
            d['dataset_path'] = os.path.join(save_dir, str(hex_dig)+'.jpg')
            d['hash_key'] = hex_dig
            d['hash_val'] = new_path
            lookup_table[d['hash_key']] = d['hash_val'] 
            old_path = os.path.join(bag_path, 'frames', d['frame_path'].split('-')[-1])

            for log in [x for x in csv_logger.as_matrix() if os.path.join(bag_path, 'frames', x[6].rstrip('/')) == old_path.rstrip('/')]:
                d['bag_name'] = log[0]
                d['time_sec'] = log[1]
                d['time_readable'] = time.ctime(int(log[1]))

                d['time_nsec'] = log[2]
                d['GPS'] = log[3]
                d['v-ego'] = log[4]
                                    
                d['key_event'] = log[5]
                d['frame'] = log[6]

            if d.get('frame', None): img_dict['frames'].append(d)
        if img_dict['frames']: img_data.append((bag_path,img_dict))

pickle_file = '/media/dean/datastore/datasets/kache_ai/frames/kacheai_logs.pickle'
pickle_dict = {'img_data':img_data,'lookup_table':lookup_table}
print('Saving to Pickle File:', pickle_file)
with open(pickle_file,"wb") as pickle_out:
    pickle.dump(pickle_dict, pickle_out)



/home/dean/miniconda3/envs/ros-kache/lib/python3.6/site-packages/ipykernel/__main__.py:36: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Saving to Pickle File: /media/dean/datastore/datasets/kache_ai/frames/kacheai_logs.pickle


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
print(img_data[5])

('/media/dean/deans_data/construction_frames/vidlog-18-09-28', {'bag_path': '/media/dean/deans_data/construction_frames/vidlog-18-09-28', 'frames': [{'frame_path': '/media/dean/deans_data/construction_frames/vidlog-18-09-28/frames/frames-frames-frame0005.jpg', 'dataset_path': '/media/dean/deans_data/construction_frames/kache_set/626c8348a139d312f94b4183bb019b7608d6c801.jpg', 'hash_key': '626c8348a139d312f94b4183bb019b7608d6c801', 'hash_val': 'vidlog-18-09-28-frames-frames-frame0005.jpg', 'bag_name': 'bagfile_18-09-28_1529_1.bag', 'time_sec': 1538174201, 'time_readable': 'Fri Sep 28 15:36:41 2018', 'time_nsec': 440282106, 'GPS': '$GNGGA,223641.40,3741.66247,N,12223.50050,W,1,12,0.72,-3.7,M,-29.8,M,,*55\r', 'v-ego': 26.26666641235352, 'key_event': 'c', 'frame': 'frame0005.jpg'}, {'frame_path': '/media/dean/deans_data/construction_frames/vidlog-18-09-28/frames/frames-frame0003.jpg', 'dataset_path': '/media/dean/deans_data/construction_frames/kache_set/b016c08d8ba018dfd9621ae681567a9aad769

In [11]:
from utils import datasets
WORKING_DIR = '/media/dean/datastore/datasets/kache_ai'
kache_set = datasets.DataFormatter(annotations_list = '/media/dean/datastore/datasets/kache_ai/frames_dev/kacheai_logs.pickle', image_list = '/media/dean/datastore/datasets/kache_ai/frames_dev/', input_format = datasets.Format.kache,
                                 output_path = os.path.join(WORKING_DIR,'scalabel_files_dev'),
                                 trainer_prefix = 'COCO_train2014_0000', 
                                 s3_bucket = 'kache-scalabel/kache_ai/frames/', check_s3 = False)

Saving to Pickle File: kache_ai_frames_dev_kacheai_logs.pickle.pickle
Length of COCO Images 24
Length of COCO Annotations: 0
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

########### DATASET DISTRIBUTION: ###########


################################################



In [13]:
kache_set._images

{'COCO_train2014_00000a7d17d665ff6c1b6010e251e88adc0aa00fdb01.jpg': {'url': 'https://s3-us-west-2.amazonaws.com/kache-scalabel/kache_ai/frames/0a7d17d665ff6c1b6010e251e88adc0aa00fdb01.jpg',
  'name': 'https://s3-us-west-2.amazonaws.com/kache-scalabel/kache_ai/frames/0a7d17d665ff6c1b6010e251e88adc0aa00fdb01.jpg',
  'coco_path': '/media/dean/deans_data/construction_frames/kache_set/0a7d17d665ff6c1b6010e251e88adc0aa00fdb01.jpg',
  'width': 1280,
  'height': 1080,
  'labels': [],
  'index': 15,
  'timestamp': 1536275810,
  'videoName': 'bagfile_18-09-06_1612_0.bag',
  'attributes': {'weather': None, 'scene': 'highway', 'timeofday': 'night'}},
 'COCO_train2014_00000a87f2e5e4e282bda1102e9675bd32c8c4546ad9.jpg': {'url': 'https://s3-us-west-2.amazonaws.com/kache-scalabel/kache_ai/frames/0a87f2e5e4e282bda1102e9675bd32c8c4546ad9.jpg',
  'name': 'https://s3-us-west-2.amazonaws.com/kache-scalabel/kache_ai/frames/0a87f2e5e4e282bda1102e9675bd32c8c4546ad9.jpg',
  'coco_path': '/media/dean/deans_data/